In [293]:
import json
import re
try:  
    from asciitree import LeftAligned
    from collections import OrderedDict as OD
except ModuleNotFoundError:
    print("Error -- Use the requirement.txt to install the dependencies")

### Listing the top 10 files in the directory tables

In [187]:
dirpath = os.path.join( os.getcwd() , "tables")
files = os.listdir(path=dirpath)[:]

schema_table= []
for file in files:
    schema_table.append ((file.split(".")[0],file.split(".")[1]))
schema_table.sort(key=lambda x: x[0])
    





crosscheck.games
crosscheck.calibration_maintenance
crosscheck.tags
games.metadata
games.vision
games.oem
games.vision_oem
games.autorecalibration
games.latency
games.live_final
games.nulls
locations.latency
locations.vision_oem
report.scheduled_no_exceptions_hist
report.hardball_pitchers
report.delivered_new_pk
report.delivered_exceptions_hist
report.verified_levels_18
report.delivered_no_exceptions
report.delivered_combined_hist
report.practice_pitching
report.false_delivered_hist
report.mlb_orgs
report.schedule_new_level
report.scheduled_no_exceptions
report.exp_access_rules_hist
report.delivered_no_exceptions_hist
report.scheduled_combined
report.practice_pitching_dev
report.exp_access_rules
report.scheduled_exceptions
rundown.access_rules_history
rundown.location_history
rundown.access_rules_rows
rundown.location_history_rows
scout.tags
trout.app_angle


In [464]:
class CreateBranch:
    def __init__(self,filepath):
        self.parentNode = None
        self.parentNodeList = []
        self.from_text= None
        self.tables_list = []
        self.tree_dict = {}  
        self.starting_source = None
        self.joining_source= None
        self.reached_leaf = False
        
        

        self.getParentNode(filepath)
        
        if self.reached_leaf == False:
            self.getTheFromClauseText(filepath)
            self.getTheFirstSource()
            self.getTheJoinTablesList()
            self.consolidateTableList()
        
        self.construct_tree()
        
    def getParentNode(self,filepath):
        file_exists = os.path.isfile(filepath)
        self.parentNode = ".".join(os.path.basename(filepath).split(".")[:-1])
        if file_exists:
            self.parentNodeList.append(self.parentNode)
        else:
            self.reached_leaf = True
            
        
    
    def construct_tree(self):
        if self.reached_leaf == False:
            
            t_list=[]
            for i in self.tables_list:
                t_list.append((i,{}))
            self.tree_dict[self.parentNode] = OD(t_list)
        else:
            self.tree_dict[self.parentNode] = {}
#             pass
        tr = LeftAligned()
        print(tr(self.tree_dict))
    
    
    def getTheFromClauseText(self,filepath):
        try:
            fileObj = open(filepath)
            config_dict = json.load(fileObj)
            try:
                # Fetch the from clause
                self.from_text = config_dict["query"]["L"][0]["M"]["from"]["S"] # For query with list
            except:
                try:
                    self.from_text = config_dict["query"]["M"]["from"]["S"] # Single query without list
                except:
                    # Do nothing if this fails too, and return from_text as None
                    pass
            fileObj.close()
        except FileNotFoundError:
            return None
        
    def getTheFirstSource(self):
        
        if self.from_text != None:

            p = re.compile(r"""
                (?P<starting>\w+[.]\w+) # Get the starting text    
                """, re.VERBOSE)
            
            m = p.search(self.from_text)
            self.starting_source = m.group('starting')
            
    def getTheJoinTablesList(self):
        if self.from_text !=None:
            p = re.compile(r"""
            join\s+(?P<table>\w+[.]\w+)
            """, re.VERBOSE)
            self.joining_source = p.findall(self.from_text)
            
    def consolidateTableList(self):
        self.tables_list.append(self.starting_source)
        self.tables_list= self.tables_list + self.joining_source
        
            
filepath = os.path.join(dirpath,'report.practice_pitching_dev.json')
# filepath = os.path.join(dirpath,'scout.teams.json')
c = CreateBranch(filepath)


report.practice_pitching_dev
 +-- practice.plays_dev
 +-- practice.games_dev


In [434]:
c.tree_dict

{'report.practice_pitching_dev': OrderedDict([('practice.plays_dev', {}),
              ('practice.games_dev', {})])}

In [435]:
c.tables_list

['practice.plays_dev', 'practice.games_dev']

In [436]:
c.parentNode

'report.practice_pitching_dev'

In [437]:
c.parentNodeList

['report.practice_pitching_dev']

In [468]:
def getFilepath (filename):
    filepath = os.path.join(dirpath,filename+".json")
    return filepath

filepath = os.path.join(dirpath,'games.oem.json')

treeDict = {}


def recursiveFunction(filepath):
    tempObj = CreateBranch(filepath)
    if tempObj.reached_leaf == False:
        for item in tempObj.tables_list:
            recursiveFunction(getFilepath(item))
    else:
        pass
    

recursiveFunction(filepath)

games.oem
 +-- radar.measurement
 +-- radar.live_pitch_data
 +-- crosscheck.tags
radar.measurement
radar.live_pitch_data
crosscheck.tags
 +-- scout.tags
 +-- base.games
scout.tags
 +-- base.tags
 +-- dict.player_dedup
base.tags
dict.player_dedup
dict.player_dedup
base.games
